In [13]:
import datetime as dt
from datetime import datetime
from secrets_p5 import secrets
import json
import pandas as pd
from sqlalchemy import create_engine, text
import requests

In [153]:
def get_data_from_api(**kwarg):
    api_key = kwarg['api_key']
    api_endpoint = kwarg['api_endpoint']
    nickname = kwarg['nickname']
    cohort = kwarg['cohort']
    api_method = kwarg['api_method']
    offset = 0
    headers = {'X-Nickname': nickname, 
               'X-Cohort': cohort, 
               'X-API-KEY': api_key
           }
    params = kwarg['params']
    start_time = dt.datetime.now()
    while (dt.datetime.now() - start_time).total_seconds() < 300:
        res = requests.get('https://'+api_endpoint + api_method,
                           headers = headers, 
                           params = params)
        res_json = json.loads(res.content)
        print(f'offset = {offset}, len(res_json) = {len(res_json)}')
        offset = offset + len(res_json)
        params['offset'] = offset
        if len(res_json) == 0:
            break
        df = pd.DataFrame(res_json)

In [154]:
op_kwargs = {
        'nickname': secrets['nickname'],
        'cohort': secrets['cohort'],
        'api_key': secrets['api_key'],
        'api_endpoint': secrets['api_endpoint'],
        'pg_host': secrets['pg_host'],
        'pg_port': secrets['pg_port'],
        'pg_base_name': secrets['pg_base_name'],
        'pg_user': secrets['pg_user'],
        'pg_password': secrets['pg_password'],
        'stage_schema_name': secrets['stage_schema_name'],
        'dm_schema_name': secrets['dm_schema_name'],
        'api_method': '/deliveries',
        'params': {'sort_field':'date', 
                   'sort_direction':'asc',
                   'limit': 10,
                   'from': '2024-08-07 00:00:00',
                   'to': '2024-08-07 01:40:59'}
        }

In [155]:
def get_data(**kwarg):
    api_key = kwarg['api_key']
    api_endpoint = kwarg['api_endpoint']
    nickname = kwarg['nickname']
    cohort = kwarg['cohort']
    api_method = kwarg['api_method']
    offset = 10
    headers = {'X-Nickname': nickname, 
               'X-Cohort': cohort, 
               'X-API-KEY': api_key
           }
    params = kwarg['params']
    params['offset'] = offset
    res = requests.get('https://'+api_endpoint + api_method,
                           headers = headers, 
                           params = params)
    res_json = json.loads(res.content)
    return pd.DataFrame(res_json)

In [148]:
df.head(50)

""


In [157]:
import datetime

current_datetime = datetime.datetime.now()

In [159]:
from datetime import datetime, timedelta

current_datetime = datetime.now()
seven_days_ago = current_datetime - timedelta(days=7)

print(seven_days_ago)

2024-08-04 17:41:28.326684


In [ ]:
date_string = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
print(date_string)